# Keyword-Extraction using BERT

Use BERT Token Classification Model to extract keyword tokens from a sentence.

## Prepare Dataset for BERT.

Convert Sem-Eval 2010 keyword recognition dataset to BIO format dataset.

In [ ]:
import os
from nltk.tokenize import sent_tokenize

In [ ]:
train_path = "KeyWordDataSet"

In [ ]:
txt = sorted([f for f in os.listdir(train_path) if not f.endswith("-justTitle.txt") and not f.endswith(".key") and not f.endswith("-CrowdCountskey")])
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")])

In [ ]:
filekey = dict()
print(len(txt))
print(len(key))
for i, k in enumerate(txt):
    try:
       filekey[key[i]] = k
      # print(train_path + "/" + filekey[key[i]])
    except Exception as E:
      print(E)

    #print(k)

144
144


In [ ]:
def convert(key):
    sentences = ""
    
    for line in open(train_path + "/" + filekey[key], 'r'):
        sentences += (" " + line.rstrip())
    tokens = sent_tokenize(sentences)
    key_file = open(train_path + "/" + str(key),'r')
    keys = [line.strip() for line in key_file]
    key_sent = []
    labels = []
    for token in tokens:
        z = ['O'] * len(token.split())
        for k in keys:
            if k in token:
                
                if len(k.split())==1:
                    try:
                        z[token.lower().split().index(k.lower().split()[0])] = 'B'
                    except ValueError:
                        continue
                elif len(k.split())>1:
                    try:
                        if token.lower().split().index(k.lower().split()[0]) and token.lower().split().index(k.lower().split()[-1]):
                            z[token.lower().split().index(k.lower().split()[0])] = 'B'
                            for j in range(1, len(k.split())):
                                z[token.lower().split().index(k.lower().split()[j])] = 'I'
                    except ValueError:
                        continue
        for m, n in enumerate(z):
            if z[m] == 'I' and z[m-1] == 'O':
                z[m] = 'O'

        if set(z) != {'O'}:
            labels.append(z) 
            key_sent.append(token)
    return key_sent, labels

In [ ]:
import nltk
nltk.download('punkt')
sentences_ = []
labels_ = []
for key, value in filekey.items():
  print(filekey[key])
  if ".ipynb_checkpoints" not in filekey[key]:
    s, l = convert(key)
    sentences_.append(s)
    labels_.append(l)
sentences = [item for sublist in sentences_ for item in sublist]
labels = [item for sublist in labels_ for item in sublist]
# print(len(sentences), len(labels))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
C-41.txt
C-42.txt
C-44.txt
C-45.txt
C-46.txt
C-48.txt
C-49.txt
C-50.txt
C-52.txt
C-53.txt
C-54.txt
C-55.txt
C-56.txt
C-57.txt
C-58.txt
C-61.txt
C-62.txt
C-65.txt
C-66.txt
C-67.txt
C-68.txt
C-69.txt
C-71.txt
C-72.txt
C-74.txt
C-75.txt
C-76.txt
C-77.txt
C-78.txt
C-79.txt
C-80.txt
C-81.txt
C-83.txt
C-84.txt
H-35.txt
H-37.txt
H-38.txt
H-40.txt
H-41.txt
H-42.txt
H-43.txt
H-44.txt
H-45.txt
H-46.txt
H-47.txt
H-48.txt
H-49.txt
H-50.txt
H-52.txt
H-53.txt
H-54.txt
H-60.txt
H-61.txt
H-62.txt
H-63.txt
H-64.txt
H-69.txt
H-73.txt
H-77.txt
H-79.txt
H-81.txt
H-82.txt
H-83.txt
H-84.txt
H-85.txt
H-87.txt
H-88.txt
H-90.txt
H-92.txt
H-95.txt
H-96.txt
H-97.txt
H-98.txt
I-37.txt
I-38.txt
I-42.txt
I-43.txt
I-45.txt
I-46.txt
I-47.txt
I-48.txt
I-49.txt
I-50.txt
I-51.txt
I-52.txt
I-53.txt
I-54.txt
I-55.txt
I-56.txt
I-57.txt
I-58.txt
I-59.txt
I-60.txt
I-61.txt
I-62.txt
I-63.txt
I-64.txt
I-65.txt
I-66.txt
I-6

In [ ]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 4.1 MB/s 
     |████████████████████████████████| 102 kB 31.2 MB/s 
     |████████████████████████████████| 7.8 MB 42.3 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 138 kB 52.8 MB/s 
     |████████████████████████████████| 127 kB 49.3 MB/s 
  Created wheel for boto3: filename=boto3-1.18.27-py3-none-any.whl size=129020 sha256=be126aecf22a0f55b0571cba3c96bbd8a724398ac2490be5fc3b85fe0a7d5ebd
  Stored in directory: /root/.cache/pip/wheels/cc/8c/12/2458a3e4630a0c93877365dfcc6449f0b02ccafa84a2baf5f3
Successfully built boto3
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.

## Model

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [ ]:
MAX_LEN = 75
bs = 32

In [ ]:
tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

In [ ]:
device = "cpu"
n_gpu = torch.cuda.device_count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


100%|██████████| 231508/231508 [00:00<00:00, 313980.33B/s]


In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (766 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (714 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (544 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (630 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1001 > 512). Running this sequence through BERT will result in indexing errors


In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.2)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.2)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

100%|██████████| 407873900/407873900 [00:33<00:00, 12042913.68B/s]


In [ ]:
model = model.cuda()

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.1},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.1}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 956 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=d084c014217387a527ecd17cccd1aa713c2f842c7b4a3e22250ed46f3bd87761
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
epochs = 4
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    #print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import pickle
my_file = open("/content/key-bert", 'wb')
my_file = pickle.dump(model,my_file)

In [ ]:
model=pd.read_pickle(r'/content/key-bert')

.

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
#print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.2017330676317215
Validation Accuracy: 0.9566880952380953


### Get keywords from sentence

In [ ]:
def keywordextract(sentence):
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    #logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

In [ ]:
#text = "Business Wire IndiaAdding another feather to its cap, Angel Broking which enable clients to avail services digitally. Our customer outreach spans across approximately 97.6% or 18,797 pin codes in India. ABL manages"

text="Two men have been arrested on charges of cheating after a shooter was lured into selling his phone and air pistol for an exorbitant amount of cash but was given a bag containing banknote-size papers instead, the police said on Wednesday."

In [ ]:
keywordextract(text)

RuntimeError: ignored

In [ ]:
keywordextract(text2)

NameError: ignored

In [ ]:
model.s

In [ ]:
import pickle
my_file = open("/content/key-bert", 'wb')
my_file = pickle.dump(model,my_file)